# 런타임에 체인 내부 구성하기
이번 튜토리얼은 Chain 을 호출시 다양한 옵션을 동적으로 설정할 수 있는 방법을 알아보겠습니다.

다음의 2가지 방식으로 동적 구성을 할 수 있습니다.

- 첫째, configurable_fields 메서드입니다. 이 메서드를 통해 실행 가능한 객체의 특정 필드를 구성할 수 있습니다.

- 둘째, configurable_alternatives 메서드입니다. 이 메서드를 사용하면 런타임 중에 설정할 수 있는 특정 실행 가능한 객체에 대한 대안을 나열할 수 있습니다.

## configurable_fields
`configurable_fields` 는 시스템의 설정 값을 정의하는 필드를 의미합니다.

### 동적 속성 지정
`ChatOpenAI` 을 사용할 때, 우리는 `model_name` 와 같은 설정을 조정할 수 있습니다.

`model_name` 은 GPT 의 버전을 명시할 때 사용하는 속성입니다. 예를 들어, `gpt-4o`, `gpt-4o-mini` 등을 설정하여 모델을 선택할 수 있습니다.

만약, 고정된 `model_name` 이 아닌 동적으로 모델을 지정하고 싶을 때는 다음과 같이 `ConfigurableField` 를 활용하여 동적으로 설정할 수 있는 속성 값으로 변환할 수 있습니다.

In [1]:
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import ConfigurableField
from langchain_openai import ChatOpenAI

model = ChatOpenAI(temperature=0, model_name="gpt-4o")

model.invoke("대한민국의 수도는 어디야?").__dict__

{'content': '대한민국의 수도는 서울입니다. 서울은 대한민국의 정치, 경제, 문화의 중심지로서 많은 중요한 정부 기관과 기업들이 위치해 있습니다.',
 'additional_kwargs': {'refusal': None},
 'response_metadata': {'token_usage': {'completion_tokens': 33,
   'prompt_tokens': 15,
   'total_tokens': 48},
  'model_name': 'gpt-4o-2024-05-13',
  'system_fingerprint': 'fp_992d1ea92d',
  'finish_reason': 'stop',
  'logprobs': None},
 'type': 'ai',
 'name': None,
 'id': 'run-cc043bb2-0524-4e05-8aca-fe89e429d358-0',
 'example': False,
 'tool_calls': [],
 'invalid_tool_calls': [],
 'usage_metadata': {'input_tokens': 15,
  'output_tokens': 33,
  'total_tokens': 48}}

In [3]:
model = ChatOpenAI(temperature=0).configurable_fields(
    model_name=ConfigurableField(
        id="gpt_version",
        name="Version of GPT",
        description="Official model name of GPTs. ex) gpt-4o, gpt-4o-mini",
    )
)
# model.invoke() 호출시 config={"configurable": {"키": "값"}} 형식으로 동적 지정할 수 있습니다.

In [5]:
model.invoke(
    "대한민국의 수도는 어디야?",
    # gpt_version 을 gpt-3.5-turbo 로 설정합니다.
    config={"configurable": {"gpt_version": "gpt-3.5-turbo"}},
).__dict__

{'content': '대한민국의 수도는 서울이야.',
 'additional_kwargs': {'refusal': None},
 'response_metadata': {'token_usage': {'completion_tokens': 16,
   'prompt_tokens': 22,
   'total_tokens': 38},
  'model_name': 'gpt-3.5-turbo-0125',
  'system_fingerprint': None,
  'finish_reason': 'stop',
  'logprobs': None},
 'type': 'ai',
 'name': None,
 'id': 'run-f510a289-3988-4ad7-aa16-66a676e5a83a-0',
 'example': False,
 'tool_calls': [],
 'invalid_tool_calls': [],
 'usage_metadata': {'input_tokens': 22,
  'output_tokens': 16,
  'total_tokens': 38}}

In [6]:
model.invoke(
    # gpt_version 을 gpt-4o-mini 로 설정합니다.
    "대한민국의 수도는 어디야?",
    config={"configurable": {"gpt_version": "gpt-4o-mini"}},
).__dict__

{'content': '대한민국의 수도는 서울입니다.',
 'additional_kwargs': {'refusal': None},
 'response_metadata': {'token_usage': {'completion_tokens': 8,
   'prompt_tokens': 15,
   'total_tokens': 23},
  'model_name': 'gpt-4o-mini-2024-07-18',
  'system_fingerprint': 'fp_483d39d857',
  'finish_reason': 'stop',
  'logprobs': None},
 'type': 'ai',
 'name': None,
 'id': 'run-b480ebf9-b502-4709-b52d-59ac28419db2-0',
 'example': False,
 'tool_calls': [],
 'invalid_tool_calls': [],
 'usage_metadata': {'input_tokens': 15,
  'output_tokens': 8,
  'total_tokens': 23}}

In [7]:
# id와 version을 넣어준것
model.with_config(configurable={"gpt_version": "gpt-4o-mini"}).invoke(
    "대한민국의 수도는 어디야?"
).__dict__

{'content': '대한민국의 수도는 서울입니다.',
 'additional_kwargs': {'refusal': None},
 'response_metadata': {'token_usage': {'completion_tokens': 8,
   'prompt_tokens': 15,
   'total_tokens': 23},
  'model_name': 'gpt-4o-mini-2024-07-18',
  'system_fingerprint': 'fp_483d39d857',
  'finish_reason': 'stop',
  'logprobs': None},
 'type': 'ai',
 'name': None,
 'id': 'run-f36e1f35-c184-4e9b-99f1-84c3dbbaf508-0',
 'example': False,
 'tool_calls': [],
 'invalid_tool_calls': [],
 'usage_metadata': {'input_tokens': 15,
  'output_tokens': 8,
  'total_tokens': 23}}

In [9]:
# 템플릿에서 프롬프트 템플릿을 생성합니다.
prompt = PromptTemplate.from_template("{x} 보다 큰 위의 난수를 선택합니다.")
chain = (
    prompt | model
)  # 프롬프트와 모델을 연결하여 체인을 생성합니다. 프롬프트의 출력이 모델의 입력으로 전달됩니다.
chain.invoke({"x": 0}).__dict__  # 체인을 호출하고 입력 변수 "x"에 0을 전달합니다.

{'content': '5',
 'additional_kwargs': {'refusal': None},
 'response_metadata': {'token_usage': {'completion_tokens': 1,
   'prompt_tokens': 21,
   'total_tokens': 22},
  'model_name': 'gpt-3.5-turbo-0125',
  'system_fingerprint': None,
  'finish_reason': 'stop',
  'logprobs': None},
 'type': 'ai',
 'name': None,
 'id': 'run-179d455b-9e01-4eea-b588-ada1b7282842-0',
 'example': False,
 'tool_calls': [],
 'invalid_tool_calls': [],
 'usage_metadata': {'input_tokens': 21,
  'output_tokens': 1,
  'total_tokens': 22}}

In [10]:
# 체인을 호출할 때 설정을 지정하여 체인을 호출할 수 있습니다.
chain.with_config(configurable={"gpt_version": "gpt-4o"}).invoke({"x": 0}).__dict__

{'content': '위의 난수를 선택하는 방법은 여러 가지가 있습니다. 예를 들어, 파이썬(Python) 프로그래밍 언어를 사용하여 0보다 큰 난수를 선택하는 방법을 보여드리겠습니다.\n\n```python\nimport random\n\n# 0보다 큰 난수를 선택합니다.\nrandom_number = random.uniform(0.0001, 1.0)\nprint(random_number)\n```\n\n위의 코드는 `random.uniform(a, b)` 함수를 사용하여 0.0001과 1.0 사이의 난수를 생성합니다. 이 함수는 a와 b 사이의 부동 소수점 난수를 반환합니다. 0보다 큰 난수를 원하므로, 0.0001과 같은 작은 값을 사용하여 0을 피할 수 있습니다.\n\n또는, 정수 난수를 원한다면 `random.randint(a, b)` 함수를 사용할 수 있습니다.\n\n```python\nimport random\n\n# 0보다 큰 정수 난수를 선택합니다.\nrandom_integer = random.randint(1, 100)\nprint(random_integer)\n```\n\n이 코드는 1과 100 사이의 정수 난수를 반환합니다. 원하는 범위에 맞게 a와 b 값을 조정할 수 있습니다.',
 'additional_kwargs': {'refusal': None},
 'response_metadata': {'token_usage': {'completion_tokens': 255,
   'prompt_tokens': 17,
   'total_tokens': 272},
  'model_name': 'gpt-4o-2024-05-13',
  'system_fingerprint': 'fp_25624ae3a5',
  'finish_reason': 'stop',
  'logprobs': None},
 'type': 'ai',
 'name': None,
 'id': 'run-fb988464-f091-4f47-a5b8-83f31bd38c5a-0',
 'exampl

# Configurable Alternatives: Runnable 객체 자체의 대안 설정
런타임에 설정할 수 있는 Runnable 에 대한 대안을 구성합니다.

구성 가능한 대안들

`ChatAnthropic` 의 구성 가능한 언어 모델은 다양한 작업과 컨텍스트에 적용할 수 있는 유연성을 제공합니다.

동적으로 설정(Config) 값을 변경하기 위하여 모델에 설정하는 파라미터를 ConfigurableField 객체로 설정합니다.

- `model`: 사용할 기본 언어 모델을 지정합니다.

- `temperature`: 0에서 1 사이의 값으로, 샘플링의 무작위성을 제어합니다. 값이 낮을수록 더 결정적이고 반복적인 출력이 생성되며, 값이 높을수록 더 다양하고 창의적인 출력이 생성됩니다.

LLM 객체의 대안(alternatives) 설정 방법
LLM(Large Language Model)을 활용하여 이를 수행하는 방법을 살펴보겠습니다.

[참고]

- ChatAnthropic 모델을 사용하기 위하여 API KEY를 발급받아 설정해야합니다.
- 링크: https://console.anthropic.com/dashboard
- 아래 주석을 해제하고 API KEY를 설정하거나, .env 파일에 설정합니다.
ANTHROPIC_API_KEY 환경변수를 설정합니다

In [15]:
from langchain.prompts import PromptTemplate
#from langchain_anthropic import ChatAnthropic
from langchain_core.runnables import ConfigurableField
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(
    temperature=0, model="gpt-4o-mini"
).configurable_alternatives(
    # 이 필드에 id를 부여합니다.
    # 최종 실행 가능한 객체를 구성할 때, 이 id를 사용하여 이 필드를 구성할 수 있습니다.
    ConfigurableField(id="llm"),
    # 기본 키를 설정합니다.
    # 이 키를 지정하면 위에서 초기화된 기본 LLM(ChatAnthropic)이 사용됩니다.
    default_key="openai",    
    # 'gpt4'라는 이름의 새 옵션을 추가하며, 이는 `ChatOpenAI(model="gpt-4")`와 동일합니다.
    gpt4o=ChatOpenAI(model="gpt-4o"),
    # 여기에 더 많은 구성 옵션을 추가할 수 있습니다.
)
prompt = PromptTemplate.from_template("{topic} 에 대해 간단히 설명해주세요.")
chain = prompt | llm

In [16]:
# Anthropic을 기본으로 호출합니다.
chain.invoke({"topic": "뉴진스"}).__dict__

{'content': '뉴진스(NewJeans)는 2022년 데뷔한 대한민국의 걸그룹으로, 하이브의 자회사인 어도어(ADOR) 소속입니다. 그룹은 민지, 하니, 다니엘, 해린, 혜인으로 구성되어 있으며, 다양한 음악 스타일과 독창적인 콘셉트로 주목받고 있습니다. 뉴진스는 데뷔곡 "Attention"과 "Hype Boy"로 큰 인기를 얻었으며, 그들의 음악과 패션은 젊은 세대에게 많은 영향을 미치고 있습니다. 그룹은 신선한 이미지와 트렌디한 사운드로 K-pop 씬에서 빠르게 자리 잡고 있습니다.',
 'additional_kwargs': {'refusal': None},
 'response_metadata': {'token_usage': {'completion_tokens': 152,
   'prompt_tokens': 18,
   'total_tokens': 170},
  'model_name': 'gpt-4o-mini-2024-07-18',
  'system_fingerprint': 'fp_483d39d857',
  'finish_reason': 'stop',
  'logprobs': None},
 'type': 'ai',
 'name': None,
 'id': 'run-ebcc475d-af79-430c-a8cd-693ece4a9f12-0',
 'example': False,
 'tool_calls': [],
 'invalid_tool_calls': [],
 'usage_metadata': {'input_tokens': 18,
  'output_tokens': 152,
  'total_tokens': 170}}

In [17]:
# 체인의 설정을 변경하여 호출합니다.
chain.with_config(configurable={"llm": "gpt4o"}).invoke({"topic": "뉴진스"}).__dict__

{'content': '뉴진스(NewJeans)는 대한민국의 걸그룹으로, 2022년에 데뷔하였습니다. 이 그룹은 어도어(ADOR)라는 하이브 레이블 소속입니다. 뉴진스는 독특한 음악 스타일과 패션으로 주목받고 있으며, 데뷔와 동시에 큰 인기를 끌었습니다. 그룹의 이름은 "새로운 트렌드(New Jeans)"와 "영원히 사랑받는 진(Jeans)"이라는 두 가지 의미를 담고 있습니다. 멤버들은 뛰어난 가창력과 퍼포먼스로 팬들에게 사랑받고 있습니다.',
 'additional_kwargs': {'refusal': None},
 'response_metadata': {'token_usage': {'completion_tokens': 130,
   'prompt_tokens': 18,
   'total_tokens': 148},
  'model_name': 'gpt-4o-2024-05-13',
  'system_fingerprint': 'fp_25624ae3a5',
  'finish_reason': 'stop',
  'logprobs': None},
 'type': 'ai',
 'name': None,
 'id': 'run-390fa457-be3d-4b02-9567-9d196fd590bb-0',
 'example': False,
 'tool_calls': [],
 'invalid_tool_calls': [],
 'usage_metadata': {'input_tokens': 18,
  'output_tokens': 130,
  'total_tokens': 148}}

In [18]:
# 체인의 설정을 변경하여 호출합니다.
chain.with_config(configurable={"llm": "openai"}).invoke(
    {"topic": "뉴진스"}
).__dict__

{'content': '뉴진스(NewJeans)는 2022년 데뷔한 대한민국의 걸그룹으로, 하이브의 자회사인 어도어(ADOR) 소속입니다. 그룹은 민지, 하니, 다니엘, 해린, 혜인으로 구성되어 있으며, 다양한 음악 스타일과 독창적인 콘셉트로 주목받고 있습니다. 뉴진스는 데뷔곡 "Attention"과 "Hype Boy"로 큰 인기를 얻었으며, 그들의 음악은 R&B, 팝, 힙합 등 다양한 장르를 아우릅니다. 또한, 세련된 패션과 비주얼로도 많은 사랑을 받고 있습니다. 뉴진스는 빠르게 성장하며 K-pop 씬에서 중요한 위치를 차지하고 있습니다.',
 'additional_kwargs': {'refusal': None},
 'response_metadata': {'token_usage': {'completion_tokens': 168,
   'prompt_tokens': 18,
   'total_tokens': 186},
  'model_name': 'gpt-4o-mini-2024-07-18',
  'system_fingerprint': 'fp_483d39d857',
  'finish_reason': 'stop',
  'logprobs': None},
 'type': 'ai',
 'name': None,
 'id': 'run-84959b2c-862d-4bc6-87b5-8c37fe0fd60a-0',
 'example': False,
 'tool_calls': [],
 'invalid_tool_calls': [],
 'usage_metadata': {'input_tokens': 18,
  'output_tokens': 168,
  'total_tokens': 186}}